In [4]:
run Objects_and_functions.ipynb

In [31]:
# rectangular Lattice Generator
def Rect_Lattice(nx, ny, lx, ly, k, support_x, support_y, support_z):
    # Rectangular lattice with nx, and ny nodes in x-direction and y-direction respectively
    # lx and ly are dimensions of unit cell of lattice in x- and y-direction respecctively
    # Support vector indicate the vectors supported in x-, y- and z-direction
    
    nNodes = nx*ny;
    nSprings = (2*nx*ny) - nx - ny
    
    N_vec = []
    S_vec = []
    
    x = 0
    y = 0
    z = 0
    
    #Defining nodes
    for i in range(1, (nx + 1)):
        for j in range(1, (ny + 1)):
            node_temp = Node()
            node_temp.n = (nx*(i-1)) + j
            node_temp.x = x
            node_temp.y = y
            node_temp.z = z
            y = y + ly
            N_vec.insert(((nx*(i-1)) + j - 1), node_temp)
                    
        y = 0
        x = x + lx
    
    #Defining Supports
    for i in support_x:
        N_vec[i - 1].s_x = 1
    for i in support_y:
        N_vec[i - 1].s_y = 1
    for i in support_z:
        N_vec[i - 1].s_z = 1

    #Springs and connectivity
    temp_nSpring = 1
    l0x = lx
    l0y = ly
    temp_nNode = 0
    
    for i in range(1, (nx + 1)):
        for j in range(1, (ny + 1)):
            temp_nNode = (ny*(i-1)) + j
            
            if (j < ny):
                spring_temp = Spring()
                spring_temp.n = temp_nSpring
                spring_temp.k = k
                spring_temp.l0 = l0y
                spring_temp.i = temp_nNode
                spring_temp.j = temp_nNode + 1
                S_vec.insert(temp_nSpring - 1, spring_temp)
                N_vec[temp_nNode - 1].sp_con.append(temp_nSpring)
                N_vec[temp_nNode].sp_con.append(temp_nSpring)
                temp_nSpring = temp_nSpring + 1
                
            if (i < nx):
                spring_temp = Spring()
                spring_temp.n = temp_nSpring
                spring_temp.k = k
                spring_temp.l0 = l0x
                spring_temp.i = temp_nNode
                spring_temp.j = temp_nNode + ny
                S_vec.insert(temp_nSpring - 1, spring_temp)
                N_vec[temp_nNode - 1].sp_con.append(temp_nSpring)
                N_vec[temp_nNode + ny - 1].sp_con.append(temp_nSpring)
                temp_nSpring = temp_nSpring + 1
    
    if (temp_nSpring - 1) != nSprings:
        print("Number of springs do not match!!")
        
    return Lattice(nx, ny, N_vec, S_vec, support_x, support_y, support_z)

In [16]:
def add_braces_to_rectangular_lattice(lat_original, stiffness, orientation):
    
    lat = lat_original
    
    diag_springs = []
    temp_nSpring = len(lat.springs) + 1
    
    ly = lat.springs[0].l0
    lx = lat.springs[1].l0
    temp_nNode = 1
    l = np.sqrt((ly**2)+(lx**2))
    
    if(orientation == '/' or orientation == '|'):
        for i in range(1, lat.nx):
            for j in range(1, lat.ny):
                temp_nNode = (lat.ny*(i-1)) + j
                
                temp_spring = Spring()
                temp_spring.n = temp_nSpring
                temp_spring.k = stiffness
                temp_spring.l0 = l
                temp_spring.i = temp_nNode
                temp_spring.j = temp_nNode + lat.ny + 1
                
                lat.nodes[temp_nNode - 1].sp_con.append(temp_nSpring)
                lat.nodes[temp_nNode + lat.ny].sp_con.append(temp_nSpring)
                
                lat.springs.insert(temp_nSpring, temp_spring)
                temp_nSpring = temp_nSpring + 1
                
    if(orientation == '\\' or orientation == '|'):
        for i in range(2, lat.nx + 1):
            for j in range(1, lat.ny):
                temp_nNode = (lat.ny*(i-1)) + j
                
                temp_spring = Spring()
                temp_spring.n = temp_nSpring
                temp_spring.k = stiffness
                temp_spring.l0 = l
                temp_spring.i = temp_nNode
                temp_spring.j = temp_nNode - lat.ny + 1
                
                lat.nodes[temp_nNode - 1].sp_con.append(temp_nSpring)
                lat.nodes[temp_nNode - lat.ny].sp_con.append(temp_nSpring)
                
                lat.springs.insert(temp_nSpring, temp_spring)
                temp_nSpring = temp_nSpring + 1        
    
    return(lat)
            
            
            

In [4]:
def add_layers_in_z_direction(lat_original, nLayers, lz, vertical_bar_stiffness, strut_stiffness, vertical_strut_orientation):
    
    lat = copy.deepcopy(lat_original)
    
    nNodes_original = len(lat.nodes)
    nSprings_original = len(lat.springs)
    nSprings_current = nSprings_original
    nNodes_current = nNodes_original
    
    ly = lat.springs[0].l0
    lx = lat.springs[1].l0
        
    lat.nz = nLayers
    
    
    for i in range(1, nLayers):
        lat_temp = copy.deepcopy(lat_original)
        #Nodes of the new layers
        for j in lat_temp.nodes:
            j.z = j.z + i*lz
            j.n = j.n + nNodes_current
            
            for k in range(0, len(j.sp_con)):
                j.sp_con[k] = j.sp_con[k] + nSprings_current
                
            j.s_x = 0
            j.s_y = 0
            j.s_z = 0
            
        #Springs of the new layers
        for j in lat_temp.springs:
            j.n = j.n + nSprings_current
            j.i = j.i + nNodes_current
            j.j = j.j + nNodes_current
        
        nNodes_current = nNodes_current + nNodes_original
        nSprings_current = nSprings_current + nSprings_original
        
        lat.springs = copy.deepcopy(lat.springs + lat_temp.springs)
        lat.nodes = copy.deepcopy(lat.nodes + lat_temp.nodes)
            
        #Connecting the two layers
        #Vertical Springs
        #temp_nSpring = nSprings_current + 1 : Redundant variable
        for j in range(1, lat_temp.nx + 1):
            for k in range(1, lat_temp.ny + 1):
                nSprings_current = nSprings_current + 1
                temp_nNode = nNodes_current - nNodes_original + (lat_temp.ny*(j-1)) + k
                
                temp_spring = Spring()
                temp_spring.n = nSprings_current
                temp_spring.k = vertical_bar_stiffness
                temp_spring.l0 = lz
                temp_spring.i = temp_nNode
                temp_spring.j = temp_nNode - nNodes_original
                
                lat.nodes[temp_spring.i - 1].sp_con.append(nSprings_current)
                lat.nodes[temp_spring.j - 1].sp_con.append(nSprings_current)
                
                lat.springs.insert(nSprings_current, temp_spring)
                
        #Body Diagonals

        l = np.sqrt((ly**2)+(lx**2)+(lz**2))
        
        if (1 in vertical_strut_orientation): # 
            for j in range(1, lat_temp.nx):
                for k in range(1, lat_temp.ny):
                    temp_nNode = nNodes_current - 2*nNodes_original + (lat_temp.ny*(j-1)) + k
                    nSprings_current = nSprings_current + 1
                    
                    temp_spring = Spring()
                    temp_spring.n = nSprings_current
                    temp_spring.k = strut_stiffness
                    temp_spring.l0 = l
                    temp_spring.i = temp_nNode
                    temp_spring.j = temp_nNode + lat.ny + 1 + nNodes_original
                    
                    lat.nodes[temp_spring.i - 1].sp_con.append(nSprings_current)
                    lat.nodes[temp_spring.j - 1].sp_con.append(nSprings_current)
                
                    lat.springs.insert(nSprings_current, temp_spring)

                    
        if (2 in vertical_strut_orientation):
            for j in range(1, lat_temp.nx):
                for k in range(2, lat_temp.ny + 1):
                    temp_nNode = nNodes_current - 2*nNodes_original + (lat_temp.ny*(j-1)) + k
                    nSprings_current = nSprings_current + 1
                    
                    temp_spring = Spring()
                    temp_spring.n = nSprings_current
                    temp_spring.k = strut_stiffness
                    temp_spring.l0 = l
                    temp_spring.i = temp_nNode
                    temp_spring.j = temp_nNode + lat.ny -1 + nNodes_original
                    
                    lat.nodes[temp_spring.i - 1].sp_con.append(nSprings_current)
                    lat.nodes[temp_spring.j - 1].sp_con.append(nSprings_current)
                
                    lat.springs.insert(nSprings_current, temp_spring)
                    
        if (3 in vertical_strut_orientation):
            for j in range(2, lat_temp.nx + 1):
                for k in range(1, lat_temp.ny):
                    temp_nNode = nNodes_current - 2*nNodes_original + (lat_temp.ny*(j-1)) + k
                    nSprings_current = nSprings_current + 1
                    
                    temp_spring = Spring()
                    temp_spring.n = nSprings_current
                    temp_spring.k = strut_stiffness
                    temp_spring.l0 = l
                    temp_spring.i = temp_nNode
                    temp_spring.j = temp_nNode - lat.ny + 1 + nNodes_original
                    
                    lat.nodes[temp_spring.i - 1].sp_con.append(nSprings_current)
                    lat.nodes[temp_spring.j - 1].sp_con.append(nSprings_current)
                
                    lat.springs.insert(nSprings_current, temp_spring)
                    
        if (4 in vertical_strut_orientation):
            for j in range(2, lat_temp.nx + 1):
                for k in range(2, lat_temp.ny + 1):
                    temp_nNode = nNodes_current - 2*nNodes_original + (lat_temp.ny*(j-1)) + k
                    nSprings_current = nSprings_current + 1
                    
                    temp_spring = Spring()
                    temp_spring.n = nSprings_current
                    temp_spring.k = strut_stiffness
                    temp_spring.l0 = l
                    temp_spring.i = temp_nNode
                    temp_spring.j = temp_nNode - lat.ny - 1 + nNodes_original
                    
                    lat.nodes[temp_spring.i - 1].sp_con.append(nSprings_current)
                    lat.nodes[temp_spring.j - 1].sp_con.append(nSprings_current)
                
                    lat.springs.insert(nSprings_current, temp_spring)
                    
                    
    return lat

In [29]:
def Lattice_Plot(x, y, z, lattice, deformed_springs, orientation, elev, azi, z_factor, fig_name):
    
    x = np.array(x)
    y = np.array(y)
    z = np.array(z)
    
    ax = plt.gca(projection="3d")
    ax.scatter(x,y,z, c='r',s=10)
    
    #plotting springs
    for i in lattice.springs:
        ax.plot([x[i.i - 1], x[i.j - 1]], [y[i.i - 1], y[i.j - 1]], [z[i.i - 1], z[i.j - 1]], color="black")
        
    #plotting supports
    for i in lattice.support_x:
        ax.scatter(x[i - 1], y[i - 1], z[i - 1], marker = '1', s = 100, c = 'black')
        # marker '1' is tri_down, denoting support in x-direction
        
    for i in lattice.support_y:
        ax.scatter(x[i - 1], y[i - 1], z[i - 1], marker = '2', s = 100, c = 'black')
        # marker '2' is tri_up, denoting support in y-direction
        
    for i in lattice.support_z:
        ax.scatter(x[i - 1], y[i - 1], z[i - 1], marker = '^', s = 50, c = 'black')
        
    #plotting the altered string
    for i in deformed_springs:
        ax.plot([x[lattice.springs[i - 1].i - 1], x[lattice.springs[i - 1].j - 1]], [y[lattice.springs[i - 1].i - 1], y[lattice.springs[i - 1].j - 1]], [z[lattice.springs[i - 1].i - 1], z[lattice.springs[i - 1].j - 1]], color = 'orange')
        
    #Setting the view and axes limits
    max_range = np.array([x.max()-x.min(), y.max()-y.min(), z.max()-z.min()]).max() / 2.0

    mid_x = (x.max()+x.min()) * 0.5
    mid_y = (y.max()+y.min()) * 0.5
    mid_z = (z.max()+z.min()) * 0.5
    ax.set_xlim(mid_x - max_range, mid_x + max_range)
    ax.set_ylim(mid_y - max_range, mid_y + max_range)
    ax.set_zlim((mid_z - max_range)*z_factor, (mid_z + max_range)*z_factor)

    if(orientation == 'XY'):
        elev = 90
        azi = -90
    elif(orientation == 'XZ'):
        elev = 270
        azi = 270
    elif(orientation == 'YZ'):
        elev = 0
        azi = 0
        
    
        
    ax.set_axis_off()
    #ax.set_frame_on(False)
    ax.view_init(elev, azi)
    #plt.savefig('destination_path.eps', format='eps')
    ax.grid(False)
    ax.mouse_init(1, 3)
    #ax.set_top_view()
    if (len(fig_name) != 0):
        plt.savefig((fig_name + '.png'), dpi=300)

    
    return ax

In [7]:
def Energy(Coordinates, f, M):
    nNodes = int(len(Coordinates)/3)
    x_vec = Coordinates[0: nNodes]
    y_vec = Coordinates[nNodes: 2*nNodes]
    z_vec = Coordinates[2*nNodes: 3*nNodes]

    #energy = 0    
    energy = np.dot(f, np.transpose(Coordinates))    
    for s in range(0, len(M.springs)):
        energy = energy + (0.5*(M.springs[s].k)*((Length(x_vec[M.springs[s].i - 1], y_vec[M.springs[s].i - 1], z_vec[M.springs[s].i - 1], x_vec[M.springs[s].j - 1], y_vec[M.springs[s].j - 1], z_vec[M.springs[s].j - 1]) - M.springs[s].l0)**2))
        
    return energy

In [2]:
def Energy2(x_vec, y_vec, z_vec):
    nNodes = int(len(x_vec))

    energy = 0    
    for i in Lat_Obj.springs:
        energy = energy + (0.5*(i.k)*((Length(x_vec[i.i - 1], y_vec[i.i - 1], z_vec[i.i - 1], x_vec[i.j - 1], y_vec[i.j - 1], z_vec[i.j - 1]) - i.l0)**2))
    return energy

In [1]:
def Jacobian(Coordinates, f, M):
    nNodes = int(len(Coordinates)/3)
    x_vec = Coordinates[0: nNodes]
    y_vec = Coordinates[nNodes: 2*nNodes]
    z_vec = Coordinates[2*nNodes: 3*nNodes]
    
    g_x = copy.deepcopy(f[0: nNodes])
    g_y = copy.deepcopy(f[nNodes: 2*nNodes])
    g_z = copy.deepcopy(f[2*nNodes: 3*nNodes])
    
    for i in range(1, nNodes + 1):
        for j in M.nodes[i - 1].sp_con:
            if(M.springs[j - 1].i == i):
                x = x_vec[M.nodes[M.springs[j - 1].i - 1].n - 1]
                y = y_vec[M.nodes[M.springs[j - 1].i - 1].n - 1]
                z = z_vec[M.nodes[M.springs[j - 1].i - 1].n - 1]
                x0 = x_vec[M.nodes[M.springs[j - 1].j - 1].n - 1]
                y0 = y_vec[M.nodes[M.springs[j - 1].j - 1].n - 1]
                z0 = z_vec[M.nodes[M.springs[j - 1].j - 1].n - 1]
                
            else:
                x = x_vec[M.nodes[M.springs[j - 1].j - 1].n - 1]
                y = y_vec[M.nodes[M.springs[j - 1].j - 1].n - 1]
                z = z_vec[M.nodes[M.springs[j - 1].j - 1].n - 1]
                x0 = x_vec[M.nodes[M.springs[j - 1].i - 1].n - 1]
                y0 = y_vec[M.nodes[M.springs[j - 1].i - 1].n - 1]
                z0 = z_vec[M.nodes[M.springs[j - 1].i - 1].n - 1]
            Spring_length = Length(x, y, z, x0, y0, z0)
            g_x[i - 1] = g_x[i - 1] + ((M.springs[j - 1].k)*(x - x0)*(1 - (M.springs[j - 1].l0/Spring_length)))
            g_y[i - 1] = g_y[i - 1] + ((M.springs[j - 1].k)*(y - y0)*(1 - (M.springs[j - 1].l0/Spring_length)))
            g_z[i - 1] = g_z[i - 1] + ((M.springs[j - 1].k)*(z - z0)*(1 - (M.springs[j - 1].l0/Spring_length)))
            
    
    Jac = np.concatenate((g_x, g_y, g_z), axis = 0)
    return Jac
                

In [14]:
def Jacobian_FD(coordinates, f, M):
    JacFD = np.zeros(len(coordinates))
    epsilon = 0.005
    for i in range(0, len(coordinates)):
        coord1 = copy.deepcopy(coordinates)
        coord2 = copy.deepcopy(coordinates)
        coord1[i] = coord1[i] + epsilon;
        coord2[i] = coord2[i] - epsilon;
        JacFD[i] = (Energy(coord1, f, M) - Energy(coord2, f, M))/(2*epsilon)
        
    return JacFD

In [16]:
def Point_load_at_centre(Magnitude, Lattice):
    nx = Lattice.nx
    ny = Lattice.ny
    nz = Lattice.nz
    nNodes = nx*ny*nz
    fx = np.zeros(nNodes)
    fy = np.zeros(nNodes)
    fz = np.zeros(nNodes)
    if((nx%2) == 0):
        if((ny%2) == 0):            
            nodeV1H1 = (nx*ny*(nz - 1)) + (ny*((nx/2) - 1)) + (ny/2)
            nodeV1H2 = (nx*ny*(nz - 1)) + (ny*((nx/2) - 1)) + (ny/2) + 1
            nodeV2H1 = (nx*ny*(nz - 1)) + (ny*(nx/2)) + (ny/2)
            nodeV2H2 = (nx*ny*(nz - 1)) + (ny*(nx/2)) + (ny/2) + 1
            fz[int(nodeV1H1) - 1] = Magnitude/4
            fz[int(nodeV1H2) - 1] = Magnitude/4
            fz[int(nodeV2H1) - 1] = Magnitude/4
            fz[int(nodeV2H2) - 1] = Magnitude/4
        else:
            nodeV1H = (nx*ny*(nz - 1)) + (ny*((nx/2) - 1)) + ((ny + 1)/2)
            nodeV2H = (nx*ny*(nz - 1)) + (ny*(nx/2)) + ((ny + 1)/2)
            fz[int(nodeV1H) - 1] = Magnitude/2
            fz[int(nodeV2H) - 1] = Magnitude/2
            
    else:
        if((ny%2) == 0):
            nodeVH1 = (nx*ny*(nz - 1)) + (ny*((nx - 1)/2)) + (ny/2)
            nodeVH2 = (nx*ny*(nz - 1)) + (ny*((nx - 1)/2)) + (ny/2) + 1
            fz[int(nodeVH1) - 1] = Magnitude/2
            fz[int(nodeVH2) - 1] = Magnitude/2
        else:
            nodeVH = (nx*ny*(nz - 1)) + (ny*((nx - 1)/2)) + ((ny + 1)/2)
            fz[int(nodeVH) - 1] = Magnitude
            
    f = np.concatenate((fx, fy, fz), axis = 0)
    return f
        
    